# Profesional I
## Analítica
### Departamento de Analítica y Gestión de Información
#### Prueba Específica - Junio 2021

Uno de los principales inconvenientes en la masificación de los productos de crédito móviles es el conocimiento del ingreso real del cliente. Para este tipo de canales es de alta importancia la información que se puede capturar en el momento de la solicitud, en
donde se puede pedir al cliente diligenciar una serie de variables demográficas (edad, género, estrato, empleador, etc.) y su ingreso mensual. 

Se ha identificado que existe una población de clientes que declara niveles de ingresos mucho mayores a los que realmente devengan, para los cuales es necesario construir una política que permita ajustar su endeudamiento.

Su objetivo es utilizar la base de datos proporcionada para determinar cuáles son los factores que influyen en que un cliente declare un ingreso superior al realmente devengado, y plantear recomendaciones accionables basadas en sus hallazgos.

Adicional al diagnóstico arriba mencionado, usted deberá construir un modelo de clasificación que permita identificar a los clientes que con mayor probabilidad estarían declarando un ingreso superior al real (utilice 20% como umbral para definir que el ingreso fue mayor).


Para solucionar el problema se plantea el siguiente esquema:
    
**1.** Realice labores de limpieza, exploración, y visualización de la información, que le permitan llegar a conclusiones preliminares y responder preguntas de tipo descriptivo (regiones con mayor desviación, estratos con mayor desviación, edades con mayor desviación, etc.)

**2.** Construya un modelo analítico que permita determinar si un cliente declara ingresos superiores a los reales. Explique de qué manera escogió el modelo y qué métricas utilizó para validar su desempeño.

**3.** Explique a manera de conclusión como se puede utilizar el resultado del modelo planteado, y cómo se podrían crear polìticas que mitiguen el riesgo de estos clientes.

*Recuerde que las conclusiones y/o recomendaciones deben ser accionables, es decir, deben generar un posible plan de acción que permita mejorar el resultado encontrado.*

## Solución

In [7]:
#!pip install xlrd
# Import pandas
import pandas as pd
# ... YOUR CODE FOR TASK 1 ...

# Load dataset
cc_analytics = pd.read_excel("AnexoProfesionalIAnalítica.xlsx")

# Inspect data
# ... YOUR CODE FOR TASK 1 ...
cc_analytics.head()

,id,tipo_p,CANAL,EDAD,TIPO_CLIENTE_CUENTAS,INGRESO_DECLARADO,INGRESO_REAL,ACIERTA,NIVEL_RIESGO,TIPO_VENTA,SUCURSAL,REGIONAL
0,1,Independiente,WEB,41-50,Antiguo,400000.0,1250.000,675,MEDIO,Asistido,BOYACA,CENTRO SUR
1,2,Independiente,APP,31-35,Antiguo,150000.0,781.242,717,MEDIO,Asistido,BOGOTA,BOGOTA Y CUNDINAMARCA
2,3,Independiente,APP,36-40,Nuevo Act Dav,140000.0,785.979,753,MEDIO,Asistido,BOGOTA,BOGOTA Y CUNDINAMARCA
3,4,Independiente,WEB,41-50,Antiguo,150000.0,1135.000,849,BAJO,Asistido,BOGOTA,BOGOTA Y CUNDINAMARCA
4,5,Independiente,APP,36-40,Antiguo,90000.0,781.242,653,MEDIO,Asistido,VALLE,VALLE


In [11]:
# Print summary statistics
cc_analytics_description = cc_analytics.describe()
print(cc_analytics_description)

print("\n")

# Print DataFrame information
cc_analytics_info = cc_analytics.info()
print(cc_analytics_info)

print("\n")

# Inspect missing values in the dataset
# ... YOUR CODE FOR TASK 2 ...
cc_analytics.tail(17)

                id  INGRESO_DECLARADO  INGRESO_REAL      ACIERTA
count  5333.000000        5333.000000   5333.000000  5333.000000
mean   2667.000000        8667.895975   4302.222439   753.593475
std    1539.648824       13502.841959   4928.627632    84.908766
min       1.000000           0.004000    737.717000   457.000000
25%    1334.000000        3343.042000    902.780000   687.000000
50%    2667.000000        5400.000000   2210.000000   746.000000
75%    4000.000000        9900.000000   5690.042000   812.000000
max    5333.000000      400000.000000  54481.000000   999.000000


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5333 entries, 0 to 5332
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    5333 non-null   int64  
 1   tipo_p                5333 non-null   object 
 2   CANAL                 5333 non-null   object 
 3   EDAD                  5333 non-null   object 


,id,tipo_p,CANAL,EDAD,TIPO_CLIENTE_CUENTAS,INGRESO_DECLARADO,INGRESO_REAL,ACIERTA,NIVEL_RIESGO,TIPO_VENTA,SUCURSAL,REGIONAL
5316,5317,Empleado,WEB,27-30,Nuevo Act Dav,1200.000,10896.200,763,MEDIO,No Asistido,BOGOTA,BOGOTA Y CUNDINAMARCA
5317,5318,Empleado,APP,>50,Nuevo Act Dav,180.000,1858.913,699,MEDIO,No Asistido,BOLIVAR,CARIBE
5318,5319,Empleado,WEB,41-50,Antiguo,510.000,5459.000,749,MEDIO,Asistido,TOLIMA,CENTRO SUR
5319,5320,Empleado,APP,>50,Nuevo Act Dav_Sect,1000.000,10896.200,753,MEDIO,No Asistido,BOGOTA,BOGOTA Y CUNDINAMARCA
5320,5321,Empleado,APP,36-40,Nuevo Act Dav,1375.000,15995.834,642,ALTO,No Asistido,SANTANDER,SANTANDERES Y ARAUCA
5321,5322,Empleado,APP,41-50,Antiguo,260.000,3280.708,655,MEDIO,No Asistido,NARIÑO,CENTRO SUR
5322,5323,Empleado,APP,31-35,Antiguo,580.000,7512.281,790,MEDIO,No Asistido,SUCRE,CARIBE
5323,5324,Empleado,APP,31-35,Nuevo Act Dav,500.000,6830.398,805,BAJO,No Asistido,ATLANTICO,CARIBE
5324,5325,Empleado,APP,31-35,Nuevo Act Dav,800.000,12679.309,790,MEDIO,Asistido,HUILA,CENTRO SUR
5325,5326,Empleado,APP,31-35,Antiguo,8.000,781.242,779,MEDIO,No Asistido,ANTIOQUIA,ANTIOQUIA
